<a href="https://colab.research.google.com/github/KatBCN/PODS-Project/blob/main/ActionsAnalysis_115_116_117_Raw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from datetime import datetime
import re

In [2]:
# Import raw data from Github link
print("time of data loading: ", datetime.now())
dfRaw = pd.read_csv("https://raw.githubusercontent.com/KatBCN/PODS-Project/main/obtain_data/data/event_logs/ALLCongress_BillActions_RAW.csv")

# Import original action code dictionary from Github link
actionCode_df = pd.read_csv("https://raw.githubusercontent.com/KatBCN/PODS-Project/main/obtain_data/data/actionCode_dict.csv", sep = '\t')[['Code','Action']]
actionCode_df.columns = ['actionCode','actionName']

# Import updated action code dictionary from Github link
updatedCode_df = pd.read_csv("https://raw.githubusercontent.com/KatBCN/PODS-Project/main/obtain_data/data/updatedCodes_dict.csv", sep = ',')[['Code','Action']]
updatedCode_df.columns = ['actionCode','actionName']

time of data loading:  2021-12-21 19:10:51.958068


In [69]:
actionCode_df.shape

(93, 2)

In [70]:
updatedCode_df.shape

(146, 2)

In [4]:
# Show shape of raw data frame
dfRaw.shape

(166790, 11)

In [5]:
# Show column titles and types
dfRaw.dtypes

billTitle            object
billNumber            int64
billType             object
congress              int64
fullDate             object
actionCode           object
actionName           object
type                 object
sourceSystem/name    object
text                 object
billOriginalTitle    object
dtype: object

In [6]:
# Assign data types
dfRaw = dfRaw.astype({'billNumber':object, 'congress':object, 'fullDate':'datetime64'})

In [7]:
# Check for duplicates
sum(dfRaw.duplicated())

0

In [8]:
# Show first 5 rows of data
dfRaw.head(5)

,billTitle,billNumber,billType,congress,fullDate,actionCode,actionName,type,sourceSystem/name,text,billOriginalTitle
0,A joint resolution proposing an amendment to t...,1,SJRES,116,2019-01-03,10000,Introduced in Senate,IntroReferral,Library of Congress,Introduced in Senate,A joint resolution proposing an amendment to t...
1,A joint resolution proposing an amendment to t...,1,SJRES,116,2019-01-03,NaN,NaN,IntroReferral,Senate,Read twice and referred to the Committee on th...,A joint resolution proposing an amendment to t...
2,A joint resolution relating to a national emer...,10,SJRES,116,2019-02-28,10000,Introduced in Senate,IntroReferral,Library of Congress,Introduced in Senate,A joint resolution relating to a national emer...
3,A joint resolution relating to a national emer...,10,SJRES,116,2019-02-28,NaN,NaN,IntroReferral,Senate,Read twice and referred to the Committee on Ar...,A joint resolution relating to a national emer...
4,Prohibiting Unauthorized Military Action in Ve...,11,SJRES,116,2019-02-28,10000,Introduced in Senate,IntroReferral,Library of Congress,Introduced in Senate,NaN


In [9]:
# Create function to view summary statistics of each variable.
def mySummary(df):
    for v in df.columns:
            print ("\n" + v)
            print(df[v].describe())

In [10]:
mySummary(dfRaw)


billTitle
count                                                166790
unique                                                24805
top       Interior and Environment, Agriculture and Rura...
freq                                                    359
Name: billTitle, dtype: object

billNumber
count     166790
unique      9067
top            1
freq         400
Name: billNumber, dtype: int64

billType
count     166790
unique         4
top           HR
freq      125771
Name: billType, dtype: object

congress
count     166790
unique         3
top          116
freq       65804
Name: congress, dtype: int64

fullDate
count                  166790
unique                  21963
top       2019-01-03 00:00:00
freq                      909
first     2017-01-03 00:00:00
last      2021-12-10 00:00:00
Name: fullDate, dtype: object

actionCode
count     120831
unique       110
top       H11100
freq       22995
Name: actionCode, dtype: object

actionName
count                  120799
unique            

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """


In [11]:
# Create variable of unique actionCodes
actionCodes = dfRaw['actionCode'].unique()
actionCodes

array(['10000', nan, '14500', '17000', 'H14000', 'H15000', 'H30000',
       'H8D000', 'H35000', '8000', 'H37100', '28000', '31000', 'E20000',
       'E30000', '35000', '14000', '18000', 'H1L210', 'H1L220', 'H38310',
       'H11100', '5500', 'H12300', 'H30200', '36000', 'E40000', 'H36100',
       'H8A000', 'H36110', '1000', 'Intro-H', '5000', 'H12200', 'H12410',
       'H32020', 'H32400', 'H32340', 'H32341', 'H32700', 'H32050',
       'H32600', 'H34400', 'B00100', 'H30300', 'H37300', 'H12440',
       'H37220', 'H38800', 'H12210', '19500', 'H40150', 'H41400',
       'H41610', 'H41931', '20500', 'H81000', 'H36800', 'H82000',
       'H36810', 'H40140', 'H40110', 'H37210', 'H1B000', 'H12430',
       'H38900', 'H40142', 'H11200', 'H11210', 'H12420', 'H12100',
       'H36200', 'H36210', 'H17000', 'H38300', 'H30800', '14900', '9000',
       'H41800', '20900', 'H25200', '20800', 'H40200', 'H42300', 'H40210',
       '21000', 'H42510', 'H42831', '23000', 'H29800', '32000', 'H40300',
       'H4341

In [12]:
# Show number of unique actionCodes in RawData
actionCodes.shape

(111,)

In [13]:
groupedSource = dfRaw.groupby(["sourceSystem/name"])
groupedSource.type.value_counts(normalize=True)*100

sourceSystem/name        type                
House committee actions  Committee               100.000000
House floor actions      IntroReferral            46.337151
                         Floor                    42.734579
                         Committee                 3.327306
                         Calendars                 3.220816
                         President                 2.057464
                         Discharge                 1.217601
                         ResolvingDifferences      1.093028
                         Veto                      0.012055
Library of Congress      IntroReferral            84.264224
                         Committee                 5.610673
                         Floor                     5.184278
                         BecameLaw                 2.404976
                         President                 2.212184
                         NotUsed                   0.253303
                         ResolvingDifferences      0.0

In [15]:
groupedBillType = dfRaw.groupby(["billType"])
groupedBillType.type.value_counts(normalize=True)*100

billType  type                
HJRES     IntroReferral           51.509054
          Floor                   28.672032
          Committee               10.010060
          President                4.426559
          BecameLaw                2.716298
          ResolvingDifferences     1.006036
          Calendars                0.804829
          Discharge                0.503018
          Veto                     0.301811
          NotUsed                  0.050302
HR        IntroReferral           56.637858
          Committee               20.843438
          Floor                   17.584340
          Calendars                1.610864
          President                1.277719
          BecameLaw                0.849957
          Discharge                0.605863
          ResolvingDifferences     0.482623
          NotUsed                  0.100182
          Veto                     0.007156
S         IntroReferral           64.826311
          Floor                   15.601638
 

In order to fill in missing action codes and maintain consistency, we referenced the websites: https://www.congress.gov/help/field-values/action-codes and https://github.com/usgpo/bill-status/blob/master/BILLSTATUS-XML_User_User-Guide.md#3-action-code-element-possible-values


The first round of action code analysis for the 117th congress was done in a separate notebook: https://colab.research.google.com/drive/16_phrhCTrIEBrkLjeUEfpHwuzKkm0IWO?usp=sharing


The following dictionary contains codes and names that were sourced from the websites, supplemented with our own codes and names that were extracted from the "text" portion of the action.

In [16]:
# Create dicitionary of original action codes being used
actionCode_dict = dict(zip(actionCode_df.actionCode, actionCode_df.actionName))
len(actionCode_dict)

93

In [17]:
# Display action code dictionary
actionCode_dict

{'1000': 'Introduced in House',
 '10000': 'Introduced in Senate',
 '11000': 'Referred to Senate committee',
 '12000': 'Referred to Senate subcommittee',
 '13000': 'Senate committee/subcommittee actions',
 '13100': 'Senate committee/subcommittee hearings',
 '13200': 'Senate committee/subcommittee markups',
 '13900': 'Senate committee time extension',
 '14000': 'Reported to Senate',
 '14500': 'Senate committee discharged',
 '14900': 'Senate committee report filed after reporting',
 '16000': 'Senate floor actions',
 '17000': 'Passed/agreed to in Senate',
 '18000': 'Failed of passage/not agreed to in Senate',
 '19000': 'Resolving differences -- House actions',
 '2000': 'Referred to House committee',
 '20000': 'Resolving differences -- Senate actions',
 '20800': 'Conference committee actions',
 '20900': 'Conference report filed',
 '21000': 'Conference report agreed to in House',
 '22000': 'Conference report disagreed to in House',
 '23000': 'Conference report agreed to in Senate',
 '24000':

Our goal is to fill in action codes and names that are missing.

In [18]:
# Show number of rows which are missing actionCodes in Raw Data
numMissingCodes = dfRaw.actionCode.isna().sum()
numMissingCodes

45959

In [19]:
ratioMissingCodes = numMissingCodes/dfRaw.shape[0]
ratioMissingCodes

0.2755500929312309

72% of the actions taken from the XML files have actionCodes assigned from their source databases.

In [20]:
1-ratioMissingCodes

0.7244499070687691

In [43]:
# Create subset of dataframe which only has missing action codes
dfCodeNA = dfRaw[dfRaw.actionCode.isna()] 
print(dfCodeNA.shape)
# Create subset of with no missing action codes
dfCode = dfRaw[-dfRaw.actionCode.isna()] 
print (dfCode.shape)

(45959, 11)
(120831, 11)


In [22]:
dfRaw.type.isna().sum()

0

In [23]:
len(dfRaw.type.unique()) # number of unique action types

10

In [24]:
dfRaw.type.value_counts()

IntroReferral           97160
Committee               30955
Floor                   29116
Calendars                3230
President                2596
BecameLaw                1709
Discharge                1076
ResolvingDifferences      728
NotUsed                   180
Veto                       40
Name: type, dtype: int64

In [25]:
dfCodeNA.type.value_counts()

Committee               25312
IntroReferral           14219
Floor                    4163
Calendars                1627
Discharge                 470
ResolvingDifferences      144
Veto                       24
Name: type, dtype: int64

100% of the data from the House committee actions system and the Senate system are missing action codes.


In [26]:
dfCodeNA['sourceSystem/name'].value_counts()

Senate                     24132
House committee actions    21827
Name: sourceSystem/name, dtype: int64

In [27]:
# Subset raw data frame per unique source system
dfLOC = dfRaw.loc[dfRaw['sourceSystem/name'] == 'Library of Congress']
dfHF = dfRaw.loc[dfRaw['sourceSystem/name'] == 'House floor actions']
dfHC = dfRaw.loc[dfRaw['sourceSystem/name'] == 'House committee actions']
dfS = dfRaw.loc[dfRaw['sourceSystem/name'] == 'Senate']
print("LOC: ", dfLOC.shape)
print("House Floor: ", dfHF.shape)
print("House Committee: ", dfHC.shape)
print("Senate: ", dfS.shape)

LOC:  (71061, 11)
House Floor:  (49770, 11)
House Committee:  (21827, 11)
Senate:  (24132, 11)


In [45]:
len(dfCode.actionCode.unique())

110

In [48]:
grouped = dfCode.groupby(["sourceSystem/name"])
print(grouped.actionCode.unique())
for e in grouped.actionCode.unique():
  print (len(e))

sourceSystem/name
House floor actions    [H14000, H15000, H30000, H8D000, H35000, H3710...
Library of Congress    [10000, 14500, 17000, 8000, 28000, 31000, 3500...
Name: actionCode, dtype: object
82
30


In [49]:
def fillCode (row): 
  # Updated by Mateo for senate discharge type
  # Updated by Kat for senate veto type
  """
  A function to fill missing actionCodes. This can be used with
  df['actionCode'] = df.apply (lambda row: fillCode(row) if pd.isnull(row['actionCode']) else row['actionCode'], axis=1)
  This has only been tested on rows with null actionCodes.
  If rows with non-null actionCodes are used, this will not work as intended.
  After filling the actionCodes, it is recommended to apply a dictionary
  of actionCodes and actionNames to complete the data.
  """
  if row['sourceSystem/name'] == "House committee actions":
    if "referred to the subcommittee" in str.lower(row['text']):
      return '3000'  # Referred to House subcommittee
    elif "markup" in str.lower(row['text']):
      return '4200'  # House committee/subcommittee markups
    elif "mark-up" in str.lower(row['text']):
      return '4200'  # House committee/subcommittee markups
    elif "hearings" in str.lower(row['text']):
      return '4100'  # House committee/subcommittee hearings
    elif "ordered to be reported" in str.lower(row['text']):
      return '4000'  # House committee/subcommittee actions
    # Generic rule based on type for text not matching earlier rules
    elif row['type'] == "Committee":
      return '4000'  # House committee/subcommittee actions
  # Setting rules for actionCodes related to the Senate
  elif row['sourceSystem/name'] == "Senate":
    if "received in the senate" in str.lower(row['text']):
      return '10000'  # Introduced in Senate
    elif "introduced in the senate" in str.lower(row['text']):
      return '10000'  # Introduced in Senate
    elif "referred to the committee" in str.lower(row['text']):
      return '11000'  # Referred to Senate committee
    elif "passed senate" in str.lower(row['text']):
      return '17000'  # Passed/agreed to in Senate
    elif all(word in str.lower((row['text'])) for word in ["committee", "filed", "report"]):
      return '14900' # Senate committee report filed after reporting
    elif row['type'] == "IntroReferral":
      return '11000'  # Referred to Senate committee
    elif row['type'] == "ResolvingDifferences":
      return '20000' # Resolving differences -- Senate actions
    elif row['type'] == "Discharge":
      if re.match('Senate Committee.*?discharged.*?', row['text']) is not None:
        return 'H12300'
      if re.match('Motion to discharge Senate Committee*', row['text']) is not None:
        return 'H17000'
    # Veto codes added by Kat 21 Dec 2021
    elif row['type'] == "Veto":
      if "failed" in str.lower(row['text']):
        return '35000'  # Failed of passage in Senate over veto
      if "message received" in str.lower(row['text']):
        return 'SenateVetoRcvd'  # Veto received in Senate
      if "message considered" in str.lower(row['text']):
        return 'SenateVetoCon'  # Veto considered in Senate
    # Generic rule based on type for text not matching earlier rules
    elif row['type'] == "Calendars":
      return 'SenateCal' # need to define a code for Senate Calendar and add to dictionary
    elif row['type'] == "Committee":
      if "hearings" in str.lower(row['text']):
        return '13100' # Senate committee/subcommittee hearings
      elif "markup" in str.lower(row['text']):
        return '13200'  # Senate committee/subcommittee markups
      elif "mark-up" in str.lower(row['text']):
        return '13200'  # Senate committee/subcommittee markups
      # Generic rule based on type for text not matching earlier rules
      else:
        return '13000' # Senate committee/subcommittee actions
    elif row['type'] == "Floor":
      if "message on senate action sent to the house" in str.lower(row['text']):
        return '5000'  # Reported to House
      if "message on house action received in senate" in str.lower(row['text']):
        return '14000'  # Reported to Senate
      # Generic rule based on type for text not matching earlier rules
      else:
        return '16000'  # Senate floor action
  else:
    return row['actionCode']  # do nothing if source system doesn't match rules

In [50]:
# Create a copy of original raw data to fill missing actionCodes
df = dfRaw.copy()
# Use df.apply with a lambda function to fill the missing actionCodes
# It is very important only to pass rows to the function which have a null actionCode.
df['actionCode'] = df.apply (lambda row: fillCode(row) if pd.isnull(row['actionCode']) else row['actionCode'], axis=1)  

In [51]:
# Number of null actionCodes in filled data
df.actionCode.isna().sum()

0

In [62]:
view = df[df['actionCode'] == '23000']
pd.set_option('display.max_colwidth', None)
view.shape

(8, 11)

In [68]:
view = df[df['actionCode'] == '41000']
pd.set_option('display.max_colwidth', None)
view.head()

,billTitle,billNumber,billType,congress,fullDate,actionCode,actionName,type,sourceSystem/name,text,billOriginalTitle
131898,To authorize the President to award the Medal of Honor to John L. Canley for acts of valor during the Vietnam War while a member of the Marine Corps.,4641,HR,115,2018-01-29,41000,Became Private Law,BecameLaw,Library of Congress,Became Private Law No: 115-1.,To authorize the President to award the Medal of Honor to John L. Canley for acts of valor during the Vietnam War while a member of the Marine Corps.
131899,To authorize the President to award the Medal of Honor to John L. Canley for acts of valor during the Vietnam War while a member of the Marine Corps.,4641,HR,115,2018-01-29,41000,Became Private Law,BecameLaw,Library of Congress,Signed by President.,To authorize the President to award the Medal of Honor to John L. Canley for acts of valor during the Vietnam War while a member of the Marine Corps.


In [72]:
df.actionName.isna().sum()

75317

Dictionaries of Action Codes should be analyzed per source system because multiple sources may use the same codes for different types of actions. The systems are not coordinated with each other.

The next step should be to make sure that every actionCode has a reasonable actionName and standardize the names to name that are available on this website when possible: https://www.congress.gov/help/field-values/action-codes

In [73]:
pd.set_option('display.max_rows', None)
# Create a data frame showing the frequency of actionCodes
acFreq = df.actionCode.value_counts().to_frame()
acFreq = acFreq.rename(columns={"actionCode": "Frequency"})
acFreq['actionName'] = acFreq.index.map(actionCode_dict)  # index is actionCode
acFreq

,Frequency,actionName
H11100,22995,Referred to the Committee
1000,22844,Introduced in House
Intro-H,22781,Introduced in House
10000,14710,Introduced in Senate
3000,14469,Referred to House subcommittee
11000,12256,Referred to Senate committee
H8D000,5977,DEBATE
4000,3550,House committee/subcommittee actions
H30000,3258,Consideration by House
5000,2972,Reported to House


Due to time constraints, new codes that are missing action names with fewer than 5 occurrences are not being incorporated into the dictionary.

In [103]:
new_dict2 = {'H11100-A' : "Referred to Multiple Committees",
             'H36100' : "Motion to recommit to Committee",  # similar to H36200
             'H36110' : "Motion to recommit Failed", # similar to H36210
             'H32700' : "Leaves as unfinished business",
             'H32050' : "Resolved into Committee of the Whole House",
             'H32340' : "Motion to rise",
             'H32341' : "Motion to rise agreed",
             'H32020' : "Resolved into Committee of the Whole House", # similar to H32050
             'H32600' : "House rose to report",
             'H32400' : "Speaker designated Committee Chairperson",
             'H41800' : "Speaker appointed conferees",
             'H34400' : actionCode_dict['72500'], # Amendment adopted
             'H37210' : 'Further proceedings postponed', # similar to H37220
             'H40142' : 'Further proceedings postponed', # similar to H37220
             'H40200' : 'Conference report considered',	# Should occur sometime after 20900
             'SenateVetoCon' : 'Veto considered in Senate',
             'SenateVetoRcvd' : 'Veto received in Senate',
             'H42831' : 'Motion to Reconsider agreed',
             'H42510' : actionCode_dict['21000'],  # Conference Report agreed
             'H42300' : "The previous question was ordered pursuant to the rule",
             'H12430' : 'Calendar Assignment'}            

In [104]:
view = df.loc[df['actionCode'] == 'H12430']
pd.set_option('display.max_colwidth', None)
view.text

105107    Placed on the Private Calendar, Calendar No. 1.
123649    Placed on the Private Calendar, Calendar No. 3.
136492    Placed on the Private Calendar, Calendar No. 2.
140223    Placed on the Private Calendar, Calendar No. 4.
142299    Placed on the Private Calendar, Calendar No. 5.
Name: text, dtype: object

We will write a new .csv file documenting the new action code names we have created.

In [105]:
new_dict2_df = pd.DataFrame.from_dict(new_dict2, orient='index') 
new_dict2_df.index.names = ['Code']
new_dict2_df.columns = ['Action']
# Line to write file commented out
#new_dict2_df.to_csv (r'createdCodeNames2_dict.csv', index = True, header=True)

We will create a combined dictionary of existing codes and names and our new names, and then write a new .csv file documenting all of the action codes and names to use for our analysis.


In [106]:
updatedCode_dict = {**actionCode_dict, **new_dict2}
updatedCode_dict

{'1000': 'Introduced in House',
 '10000': 'Introduced in Senate',
 '11000': 'Referred to Senate committee',
 '12000': 'Referred to Senate subcommittee',
 '13000': 'Senate committee/subcommittee actions',
 '13100': 'Senate committee/subcommittee hearings',
 '13200': 'Senate committee/subcommittee markups',
 '13900': 'Senate committee time extension',
 '14000': 'Reported to Senate',
 '14500': 'Senate committee discharged',
 '14900': 'Senate committee report filed after reporting',
 '16000': 'Senate floor actions',
 '17000': 'Passed/agreed to in Senate',
 '18000': 'Failed of passage/not agreed to in Senate',
 '19000': 'Resolving differences -- House actions',
 '19500': 'Resolving differences -- House actions',
 '2000': 'Referred to House committee',
 '20000': 'Resolving differences -- Senate actions',
 '20500': 'Resolving differences -- Senate actions',
 '20800': 'Conference committee actions',
 '20900': 'Conference report filed',
 '21000': 'Conference report agreed to in House',
 '22000'

In [108]:
len(updatedCode_dict)

146

In [109]:
updatedCode_dict_df = pd.DataFrame.from_dict(updatedCode_dict, orient='index') 
updatedCode_dict_df.index.names = ['Code']
updatedCode_dict_df.columns = ['Action']
# Line to write file commented out
#updatedCode_dict_df.to_csv (r'updatedCodes_dict.csv', index = True, header=True)